# LLM-Engineering

## Data-Import from 'Feature Engineering'


In [ ]:
with Notebook():
    from FeatureEngineering import companies

## Feature Generation with local LLMs

In [ ]:
from transformers import TFAutoModelForSequenceClassification, AutoTokenizer
from tqdm.notebook import tqdm
import tensorflow as tf

tqdm.pandas()

model_name = "facebook/bart-large-mnli"
model = TFAutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)


def is_b2b(description):
    premise = description
    hypothesis = "This company provides products, services, or solutions that are specifically designed to meet the needs of other businesses, organizations, or institutions rather than individual consumers. The company’s primary focus is on enabling or supporting the operations, efficiency, or growth of its business clients by offering tailored solutions, bulk transactions, or enterprise-level offerings that are typically not relevant for private consumers or individual end-users."
    inputs = tokenizer(premise, hypothesis, return_tensors="tf", truncation=True)

    outputs = model(**inputs)
    probabilities = tf.nn.softmax(outputs.logits, axis=-1)

    entailment_score = probabilities[0][2].numpy()
    result = 1 if entailment_score > 0.6 else 0

    print(f"Description: {description[:50]}... -> Entailment Score: {entailment_score:.2f} -> B2B: {result}")
    return result


companies["B2B"] = companies["Description"].progress_apply(is_b2b)